# Домашнее задание № 2. Мешок слов

In [1]:
import math
import numpy as np
import pandas as pd
from razdel import tokenize
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tbkazakova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
stopwords_ru = stopwords.words('russian')
stopwords_ru.append('тебе')

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [4]:
data = pd.read_csv('labeled.csv')

In [5]:
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [6]:
data.toxic.value_counts(normalize=True)
# токсичных в 2 раза меньше

0.0    0.66514
1.0    0.33486
Name: toxic, dtype: float64

In [7]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [8]:
# векторизовали с дефолтной токенизацией
vect_tfidf_default = TfidfVectorizer(min_df=10, max_df=0.3)
X_deftok = vect_tfidf_default.fit_transform(train.comment)
X_deftok_test = vect_tfidf_default.transform(test.comment)
y_deftok = train.toxic.values
y_deftok_test = test.toxic.values

In [9]:
clf = LogisticRegression(C=0.1, class_weight='balanced')
clf.fit(X_deftok, y_deftok)
preds = clf.predict(X_deftok_test)
print("С дефолтной векторизацией")
print(classification_report(y_deftok_test, preds, zero_division=0))

С дефолтной векторизацией
              precision    recall  f1-score   support

         0.0       0.89      0.84      0.87       963
         1.0       0.72      0.79      0.75       479

    accuracy                           0.83      1442
   macro avg       0.80      0.82      0.81      1442
weighted avg       0.83      0.83      0.83      1442



In [10]:
# Наша токенизация
train_razdel_toks = []
for i, row in train.iterrows():
    tokens = [_.text for _ in list(tokenize(row['comment']))]
    train_razdel_toks.append(tokens)
test_razdel_toks = []
for i, row in test.iterrows():
    tokens = [_.text for _ in list(tokenize(row['comment']))]
    test_razdel_toks.append(tokens)

# Можно было то, что сверху, положить в identity_tokenizer, но я этого не сделала.

def identity_tokenizer(text):
    return text

In [11]:
vect_tfidf_razdel = TfidfVectorizer(tokenizer=identity_tokenizer,
                                    lowercase=False,
                                    min_df=10,
                                    max_df=0.3)
X_razdtok = vect_tfidf_razdel.fit_transform(train_razdel_toks)
X_razdtok_test = vect_tfidf_razdel.transform(test_razdel_toks)
y_razdtok = train.toxic.values
y_razdtok_test = test.toxic.values

In [12]:
clf = LogisticRegression(C=0.1, class_weight='balanced')
clf.fit(X_razdtok, y_razdtok)
preds = clf.predict(X_razdtok_test)
print("С векторизацией razdel")
print(classification_report(y_razdtok_test, preds, zero_division=0))

С векторизацией razdel
              precision    recall  f1-score   support

         0.0       0.88      0.82      0.85       963
         1.0       0.68      0.77      0.72       479

    accuracy                           0.80      1442
   macro avg       0.78      0.79      0.78      1442
weighted avg       0.81      0.80      0.80      1442



In [13]:
# Я думала, что результаты с razdel чуть лучше будут.
# Несколько раз перезапустила. Иногда дефолтная, а иногда токенизация razdel выигрывает.
# Значит, разница несущественная.

## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [14]:
onethird = 1/3
table = {
    'text':["я и ты", "ты и я", "я, я и только я", "только не я", "он"],
    'я':[onethird, onethird, 0.6, onethird, 0],
    'ты':[onethird, onethird, 0, 0, 0],
    'и':[onethird, onethird, 0.2, 0, 0],
    'только':[0, 0, 0.2, onethird, 0],
    'не':[0, 0, 0, onethird, 0],
    'он':[0, 0, 0, 0, 1.0]}
table = pd.DataFrame(table)
table

,text,я,ты,и,только,не,он
0,я и ты,0.333333,0.333333,0.333333,0.000000,0.000000,0.0
1,ты и я,0.333333,0.333333,0.333333,0.000000,0.000000,0.0
2,"я, я и только я",0.600000,0.000000,0.200000,0.200000,0.000000,0.0
3,только не я,0.333333,0.000000,0.000000,0.333333,0.333333,0.0
4,он,0.000000,0.000000,0.000000,0.000000,0.000000,1.0


In [15]:
words = ['я', 'ты', 'и', 'только', 'не', 'он']

In [16]:
table_tfidf = pd.DataFrame(columns=['text', 'я', 'ты', 'и', 'только', 'не', 'он'])

for s, row in table.iterrows():
    word_tfidf = [row['text']]
    for w in range(len(words)):
        tf = row[words[w]]
        df = 0
        for word_freq in list(table[words[w]]):
            if word_freq > 0:
                df += 1
        if tf == 0:
            tfidf = 0
        else:
            tfidf = tf * math.log(len(table)/df)
        word_tfidf.append(tfidf)
    table_tfidf.loc[len(table_tfidf.index)] = word_tfidf
table_tfidf

,text,я,ты,и,только,не,он
0,я и ты,0.074381,0.30543,0.170275,0.000000,0.000000,0.000000
1,ты и я,0.074381,0.30543,0.170275,0.000000,0.000000,0.000000
2,"я, я и только я",0.133886,0.00000,0.102165,0.183258,0.000000,0.000000
3,только не я,0.074381,0.00000,0.000000,0.305430,0.536479,0.000000
4,он,0.000000,0.00000,0.000000,0.000000,0.000000,1.609438


## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [17]:
# смотрим униграммы и биграммы с токенизацией от razdel
vect_tfidf_razdel = TfidfVectorizer(tokenizer=identity_tokenizer,
                                    lowercase=False,
                                    ngram_range=(1,2),
                                    min_df=5,
                                    max_df=0.3)
X_razdtok = vect_tfidf_razdel.fit_transform(train_razdel_toks)
X_razdtok_test = vect_tfidf_razdel.transform(test_razdel_toks)
y_razdtok = train.toxic.values
y_razdtok_test = test.toxic.values

In [18]:
nb = MultinomialNB(alpha=0.9, class_prior=[0.66514, 0.33486])
nb.fit(X_razdtok, y_razdtok)
preds = nb.predict(X_razdtok_test)

print(classification_report(y_razdtok_test, preds))

# Я попробовала GaussianNB, но по метрикам получалось хуже.
# А ещё я не понимаю, насколько логично использовать MultinomialNB, если у нас 2 категории.
# Может, для случая с 2мя категориями есть более хорошее NB решение...

              precision    recall  f1-score   support

         0.0       0.82      0.97      0.89       963
         1.0       0.91      0.56      0.69       479

    accuracy                           0.84      1442
   macro avg       0.87      0.77      0.79      1442
weighted avg       0.85      0.84      0.82      1442



In [19]:
probas = nb.predict_proba(X_razdtok_test)
nb_probs = Counter()
for i in range(len(test.comment)):
    nb_probs[test.comment[i]] = float(probas[i][1])
nb_probs.most_common(10)

[('По мексикански Флаг: Ублюдок, мать твою, а ну иди сюда говно собачье, решил меня поднять? Ты, засранец вонючий, мать твою, а? Ну иди сюда, попробуй меня поднять, я тебя сам подниму ублюдок, онанист чертов, будь ты проклят, иди идиот, трахать тебя и всю семью, говно собачье, жлоб вонючий, дерьмо, сука, падла, иди сюда, мерзавец, негодяй, гад, иди сюда ты - говно, ЖОПА!\n',
  0.996005534378624),
 ('Ты охуела, мразь? Я же тебя найду блять. Гридин не свинья, ты понял, петух? НЕ СМЕЙ НАЗЫВАТЬ КУЗЬМУ СВИНЬЕЙ ДЭБИЛ ГОРОХОВЫЙ!\n',
  0.9932742880330429),
 ('НУ ВОТ, ЕРОХИН УЖЕ СВАЛИЛ ЗА БУГОР, А ТЫ ВС СИДИШЬ КАК СЫЧ В СВОЕМ ПО И СИСЯНУ ДОНАТИШЬ!!\n',
  0.9916468522994305),
 ('кукарекнула то, о чем на харкаче последний сосницкий уже докторскую защитил и тысячи тредов высрал ЕБАТЬ ФУРОР НАХУЙ. ВОТ ЭТА ДА? ДА КАК МЫ СРАЗУ НЕ ДАГАДАЛИСЬ ТО ЕБАТЬ. АЙДА НА ПЕРЕДАЧУ РАССКАЖЕШЬ ЧЕ ТАМ ДА КАК Больные пиндосы. Еврокуколды рили создают впечатление какого то умственно отсталого народа имбецила.\n',
  0.9

In [20]:
vect_CountVect = CountVectorizer(ngram_range=(1, 3),
                                 min_df=5,
                                 max_df=0.2,
                                 stop_words=stopwords_ru,
                                 analyzer='char_wb')
X = vect_CountVect.fit_transform(train.comment)
X_test = vect_CountVect.transform(test.comment)
y = train.toxic.values
y_test = test.toxic.values

In [21]:
nclf = KNeighborsClassifier(n_neighbors=8, metric='cosine')
nclf.fit(X, y)
preds = nclf.predict(X_test)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.80      0.95      0.87       963
         1.0       0.85      0.53      0.65       479

    accuracy                           0.81      1442
   macro avg       0.82      0.74      0.76      1442
weighted avg       0.82      0.81      0.80      1442



In [22]:
probas = nclf.predict_proba(X_test)
nclf_probs = Counter()
for i in range(len(test.comment)):
    nclf_probs[test.comment[i]] = float(probas[i][1])
nclf_probs.most_common(10)

[('Лахтодебил, ты читать умеешь? Я же говорю жрёт дохуя, а не едет нихуя.\n',
  1.0),
 ('Ох, свежайший приём сам дурак , ты из детского сада пишешь, что ли? Воистину, нацея дигродирует.\n',
  1.0),
 ('Нахуй тебе видосы с резиновой куклой, фетишист штоле?\n', 1.0),
 ('ты долбоеб чтоле, под кроватью куколдов ищи\n', 1.0),
 ('КОНЧ, долбаёб. Их словечко. Соси хуй, я ебал твою мать.\n', 1.0),
 ('Павел Михайловский казалось бы, причём здесь хохлы\n', 1.0),
 ('ЕБАНЫЕ ПИДОРЫ, КОГДА ПОРТИРУЕТЕ ТЕНИ КОЛОССОВ?\n', 1.0),
 ('Пиздешь Опять с продленки съебался?\n', 1.0),
 ('Тиньков был пиздец как прав, что блогеры продажные. Долго до тебя доходило.\n',
  1.0),
 ('Похуй не может быть мучительно. Похуй всегда похуй\n', 1.0)]

In [24]:
# Итог:
# KNeighborsClassifier для первых 50 с чем-то реплик уверенно предлагает абсолютную токсичность. (1.0)
# Среди них есть все из 10ти от NB?
nb_10 = []
for text in nb_probs.most_common(10):
    nb_10.append(text[0])
nclf_56 = []
for text in nclf_probs.most_common(56):
    nclf_56.append(text[0])
for text in nb_10:
    if text not in nclf_56:
        print(text)

По мексикански Флаг: Ублюдок, мать твою, а ну иди сюда говно собачье, решил меня поднять? Ты, засранец вонючий, мать твою, а? Ну иди сюда, попробуй меня поднять, я тебя сам подниму ублюдок, онанист чертов, будь ты проклят, иди идиот, трахать тебя и всю семью, говно собачье, жлоб вонючий, дерьмо, сука, падла, иди сюда, мерзавец, негодяй, гад, иди сюда ты - говно, ЖОПА!

Ты охуела, мразь? Я же тебя найду блять. Гридин не свинья, ты понял, петух? НЕ СМЕЙ НАЗЫВАТЬ КУЗЬМУ СВИНЬЕЙ ДЭБИЛ ГОРОХОВЫЙ!

НУ ВОТ, ЕРОХИН УЖЕ СВАЛИЛ ЗА БУГОР, А ТЫ ВС СИДИШЬ КАК СЫЧ В СВОЕМ ПО И СИСЯНУ ДОНАТИШЬ!!

кукарекнула то, о чем на харкаче последний сосницкий уже докторскую защитил и тысячи тредов высрал ЕБАТЬ ФУРОР НАХУЙ. ВОТ ЭТА ДА? ДА КАК МЫ СРАЗУ НЕ ДАГАДАЛИСЬ ТО ЕБАТЬ. АЙДА НА ПЕРЕДАЧУ РАССКАЖЕШЬ ЧЕ ТАМ ДА КАК Больные пиндосы. Еврокуколды рили создают впечатление какого то умственно отсталого народа имбецила.

ЕСЛИ БЫ ОДНОКЛАССНИКИ НЕ РАЗБИЛИ ПЯТИКЛАШКЕ ЕБАЛО, ТО ЕГО РАЗБИЛИ БЫ СОЛДАТЫ НАТО!!!!

Бесплатный

In [25]:
# Среди 50 с чем-то самых токсичных реплик по мнению KNeighborsClassifier нет некоторых
# из топ-10 от MultinomialNB.
# ('Павел Михайловский казалось бы, причём здесь хохлы\n', 1.0) - менее токсичное, чем что-нибудь из списка выше
# MultinomialNB точнее.

## *Задание 4 (2 балла)

Для классификаторов LogisticRegression и Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.

In [26]:
vect_tfidf = TfidfVectorizer(ngram_range=(1, 2),
                             min_df=5,
                             max_df=0.2,
                             stop_words=stopwords_ru)
# хотя tfidf убирает многие стопслова сам

X = vect_tfidf.fit_transform(train.comment)
X_test = vect_tfidf.transform(test.comment)
y = train.toxic.values
y_test = test.toxic.values

In [27]:
# словарь, индексы в списке соответствуют колонкам в матрице
wordlist = [x for x in vect_tfidf.get_feature_names()]
len(wordlist)

/usr/local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


7975

In [28]:
clf = LogisticRegression(C=0.1, class_weight='balanced')
clf.fit(X, y)
preds = clf.predict(X_test)
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.90      0.85      0.87       963
         1.0       0.73      0.81      0.77       479

    accuracy                           0.83      1442
   macro avg       0.81      0.83      0.82      1442
weighted avg       0.84      0.83      0.84      1442



In [29]:
importance = clf.coef_
clf_features = Counter()
for i in range(len(wordlist)):
    clf_features[wordlist[i]] = importance[0][i]

In [30]:
clf_features.most_common(5) #.. # надо было лемматизировать

[('хохлы', 1.6188025163209192),
 ('хохлов', 1.5304765660977848),
 ('нахуй', 1.2591559014154463),
 ('блять', 1.097152757505994),
 ('блядь', 1.0728852581931798)]

In [31]:
rf = RandomForestClassifier(n_estimators=100, max_depth=20, )
rf.fit(X, y)
preds = rf.predict(X_test)
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.68      1.00      0.81       963
         1.0       1.00      0.06      0.11       479

    accuracy                           0.69      1442
   macro avg       0.84      0.53      0.46      1442
weighted avg       0.79      0.69      0.58      1442



In [32]:
importance = rf.feature_importances_
rf_features = Counter()
for i in range(len(wordlist)):
    rf_features[wordlist[i]] = importance[i]

In [33]:
rf_features.most_common(5)  # слово "очень" не очень подходит

[('хохлов', 0.03216869148827162),
 ('хохлы', 0.02559543133338842),
 ('нахуй', 0.022729095948384553),
 ('сука', 0.017434721787487926),
 ('очень', 0.01632676877595195)]